In [29]:
import wandb
import yaml
from utils import correct_type_of_entry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
from copy import deepcopy
import seaborn as sn
pd.set_option('display.max_rows', None)

In [206]:
dataset = "mnist"

if "mnist" in dataset:
    values_to_fetch = ['complement_error', 'validation_error', 'test_error', 'test_loss']
    if "mnist" == dataset:
        sweep_config = "pretraining"
    else:
        sweep_config = "default"
elif "amazon" in dataset:
    values_to_fetch = ['complement_error', 'validation_error', 'test_error','complement_loss', 'validation_loss', 'test_loss']
    sweep_config = "baseline_transformer"
else:
    values_to_fetch = ['complement_loss', 'validation_loss', 'test_loss']
    sweep_config = "forest"

In [207]:
sweep_config_name = "./configs/sweep_configs/" + sweep_config + ".yaml"
with open(sweep_config_name) as file:
    sweep_configuration = yaml.safe_load(file)
    
hps = {}
for key, item in sweep_configuration['parameters'].items():
    if item.get('values', None) is not None:
        hps[key] = correct_type_of_entry(item['values'])
size_hyperparams = tuple([len(l) for l in hps.values()])

In [208]:
api = wandb.Api()
entity = "mathieu-bazinet"
project = "baseline_" + dataset
runs = api.runs(entity + "/" + project)

In [209]:
results_matrix = np.ones(((len(values_to_fetch),) + size_hyperparams))

for run_idx in range(len(runs)):
    run = runs[run_idx]
    if run.config['model_type'] == sweep_config or run.config['model_type'] not in ['forest', 'tree']:
        for val_to_fetch_idx in range(len(values_to_fetch)):
            try:
                matrix_idx = tuple([val_to_fetch_idx] + [hps[key].index(run.config[key]) for key in hps.keys()])
            except ValueError:
                continue
            val_to_fetch = values_to_fetch[val_to_fetch_idx]
            try:
                results_matrix[matrix_idx] = run.summary[val_to_fetch]
            except KeyError:
                results_matrix[matrix_idx] = [row[val_to_fetch] for row in run.scan_history(keys=[val_to_fetch])][-1]

In [210]:
hp_list = list(hps.values())[1:]
params_product = list(product(*hp_list))
name_list = []
idx_list = []
for params in params_product:
    name = ""
    for p in params:
        name += str(p) + " "
    name_list.append(name[:-1])
    idx = ()
    for p_idx in range(len(params)):
        p_key = list(hps.keys())[1:][p_idx]
        idx += (hps[p_key].index(params[p_idx]),)
    idx_list.append(tuple(idx))

In [211]:
reshaped_matrix = results_matrix.mean(1).reshape(results_matrix.shape[0],np.prod(results_matrix.shape[2:])).T
mean_df = pd.DataFrame(reshaped_matrix, index=name_list, columns=values_to_fetch)
mean_df

,complement_error,validation_error,test_error,test_loss
0.1 50 0.01 0.1 0.001,0.000000,0.012112,0.011593,0.062514
0.1 50 0.01 0.1 0.005,0.000265,0.015205,0.013702,0.130311
0.1 50 0.01 0.1 0.0001,0.000493,0.013495,0.013150,0.054526
0.1 50 0.01 0.2 0.001,0.000000,0.012112,0.011593,0.062514
0.1 50 0.01 0.2 0.005,0.000265,0.015205,0.013702,0.130311
0.1 50 0.01 0.2 0.0001,0.000493,0.013495,0.013150,0.054526
0.1 50 0.001 0.1 0.001,0.000000,0.009267,0.010262,0.074529
0.1 50 0.001 0.1 0.005,0.000000,0.010595,0.011648,1.040730
0.1 50 0.001 0.1 0.0001,0.000000,0.009397,0.010007,0.046151
0.1 50 0.001 0.2 0.001,0.000000,0.009148,0.010025,0.074788


In [212]:
reshaped_std = results_matrix.std(1).reshape(results_matrix.shape[0],np.prod(results_matrix.shape[2:])).T
std_df = pd.DataFrame(reshaped_std, index=name_list, columns=values_to_fetch)
std_df

,complement_error,validation_error,test_error,test_loss
0.1 50 0.01 0.1 0.001,0.000000,0.001628,0.001013,0.004125
0.1 50 0.01 0.1 0.005,0.000084,0.002549,0.000816,0.023290
0.1 50 0.01 0.1 0.0001,0.000168,0.001563,0.000338,0.003517
0.1 50 0.01 0.2 0.001,0.000000,0.001628,0.001013,0.004125
0.1 50 0.01 0.2 0.005,0.000084,0.002549,0.000816,0.023290
0.1 50 0.01 0.2 0.0001,0.000168,0.001563,0.000338,0.003517
0.1 50 0.001 0.1 0.001,0.000000,0.000590,0.001083,0.009857
0.1 50 0.001 0.1 0.005,0.000000,0.001611,0.001327,0.099633
0.1 50 0.001 0.1 0.0001,0.000000,0.000901,0.000342,0.001241
0.1 50 0.001 0.2 0.001,0.000000,0.000532,0.001333,0.009604


In [213]:
if "mnist" in dataset:
    val_of_interest = "validation_error"
elif "amazon" in dataset:
    val_of_interest = "validation_error"
else:
    val_of_interest = "validation_loss"

if "mnist" in dataset and "mnist" != dataset:
    wanted_model_type = "mlp"
    model_df = mean_df[[wanted_model_type in idx for idx in mean_df.index]]
    std_model_df = std_df[[wanted_model_type in idx for idx in std_df.index]]
    best_params = correct_type_of_entry(model_df.index[model_df[val_of_interest].argmin()].split())
    print(best_params)
    best_val_arr = model_df.loc[model_df.index[model_df[val_of_interest].argmin()]]
    std_val_arr = std_model_df.loc[model_df.index[model_df[val_of_interest].argmin()]]
else:
    best_params = correct_type_of_entry(mean_df.index[mean_df[val_of_interest].argmin()].split())
    print(best_params)
    best_val_arr = mean_df.loc[mean_df.index[mean_df[val_of_interest].argmin()]]
    std_val_arr = std_df.loc[mean_df.index[mean_df[val_of_interest].argmin()]]

if "mnist" in dataset:
    print(
    f"Complement error: {100*best_val_arr['complement_error']:.2f}±{100*std_val_arr['complement_error']:.2f} ",
    f"Validation error: {100*best_val_arr['validation_error']:.2f}±{100*std_val_arr['validation_error']:.2f} ",
      f'Test error : {100*best_val_arr['test_error']:.2f}±{100*std_val_arr['test_error']:.2f} ',
    f"Test loss : {best_val_arr['test_loss']:.4f}±{std_val_arr['test_loss']:.4f}"
)
elif "amazon" in dataset:
    print(
        f"Complement error: {100*best_val_arr['complement_error']:.2f}±{100*std_val_arr['complement_error']:.2f} ",
        f"Validation error: {100*best_val_arr['validation_error']:.2f}±{100*std_val_arr['validation_error']:.2f} ",
          f'Test error : {100*best_val_arr['test_error']:.2f}±{100*std_val_arr['test_error']:.2f} ',
        f"Complement loss: {best_val_arr['complement_loss']:.4f}±{std_val_arr['complement_loss']:.4f} ",
        f"Validation loss: {best_val_arr['validation_loss']:.4f}±{std_val_arr['validation_loss']:.4f} ",
        f"Test loss : {best_val_arr['test_loss']:.4f}±{std_val_arr['test_loss']:.4f}"
    )
else:
    print(
    f"Complement loss: {best_val_arr['complement_loss']:.4f}±{std_val_arr['complement_loss']:.4f} ",
    f"Validation loss: {best_val_arr['validation_loss']:.4f}±{std_val_arr['validation_loss']:.4f} ",
      f'Test loss : {best_val_arr['test_loss']:.4f}±{std_val_arr['test_loss']:.4f} '
)

[0.5, 50.0, 0.001, 0.1, 0.0001]
Complement error: 0.00±0.00  Validation error: 0.54±0.09  Test error : 1.08±0.09  Test loss : 0.0499±0.0042
